In [37]:
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.data_preparation import *

import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow import keras
from keras import Model, Sequential, layers, regularizers, optimizers, applications, models, callbacks
from podcast_ad_skipper.params import *
from sklearn.model_selection import train_test_split
from podcast_ad_skipper.google_cloud import *

from podcast_ad_skipper.model import *


In [38]:
bq_client = auth_gc_bigquery()
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test10"
# columns = '[spectrogram, labels]'

Running in local environment.
Authenticated successfully with BigQuery! ✅


In [39]:
rows = get_output_query_bigquery(bq_client, table_id, limit=5, columns="*")

Query returned 5 results


In [40]:
processed_columns = get_bq_processed_data(rows)

In [41]:
len(processed_columns[0])

5

In [42]:
nans_count = 0
spec_column = processed_columns[0]

def check_for_nans(spec_column, nans_count):
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
                # print(f"NaNs found in array at index {idx}")
            nans_count += 1
    print(nans_count)

In [43]:
check_for_nans(spec_column, nans_count)

0


In [44]:
X = processed_columns[0]

In [45]:
X = np.expand_dims(X, axis=-1)
X.shape

(5, 128, 216, 1)

In [46]:
X = np.array(X)

In [47]:
type(X)

numpy.ndarray

In [48]:
# for i, arr in enumerate(processed_columns[0]):
#     X = processed_columns[0]
#     if arr.shape != (128, 216):
#         print(f"Array at index {i} has shape {arr.shape}")

In [49]:
y = np.array(processed_columns[1])

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,  # This ensures similar class distribution in train/test splits
)


In [51]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [52]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [53]:
model = build_baseline_model(input_shape=(128,216,1))

In [54]:
fit_model(model, X_train, y_train, X_test, y_test)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 665ms/step - accuracy: 0.2500 - loss: 3.1921 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


(<Sequential name=sequential_1, built=True>,
 <keras.src.callbacks.history.History at 0x178871480>)

In [55]:
def save_model_to_gcs(model, bucket_name, model_name="latest_trained_model.h5"):
    """
    Save a TensorFlow model directly to Google Cloud Storage.

    Args:
        model: TensorFlow model to save
        bucket_name: Name of the GCS bucket
        model_name: Name of the model file

    Returns:
        tuple: (bool, str) - (success status, message)
    """
    # Initialize GCS client
    gc_client = auth_gc_storage()

    # Check if bucket exists
    if not gc_client.bucket(bucket_name):
        return False, f"Bucket {bucket_name} not found"

    # Get bucket
    bucket = gc_client.bucket(bucket_name)

    # Create GCS URI for saving
    gcs_uri = f"gs://{bucket_name}/{model_name}"

    # Attempt to save model directly to GCS
    save_status = models.save_model(
        model,
        gcs_uri,
        overwrite=True,
        include_optimizer=True,
        save_format='h5'
    )

    # Check save status
    if save_status is None:  # TensorFlow returns None on successful save
        print(f"Model saved successfully to {gcs_uri}")
        return True

    else:
        print("Failed to save model to GCS")
        return False

In [56]:
model_bucket = BUCKET_NAME_MODEL

In [57]:
save_model_to_gcs(model, model_bucket)

Running in local environment.
Authenticated successfully with GCS! ✅
Model saved successfully to gs://podcast-ad-skipper-model/latest_trained_model.h5


True

In [60]:
from google.api_core import retry

def save_model_to_gcs(model, bucket_name, model_name="latest_trained_model.h5",
                     timeout=300, num_retries=3):
    """
    Save a TensorFlow model to Google Cloud Storage with retry logic and configurable timeouts.

    Args:
        model: TensorFlow model to save
        bucket_name: Name of the GCS bucket
        model_name: Name of the model file
        timeout: Timeout in seconds for the upload operation (default: 300)
        num_retries: Number of retry attempts (default: 3)

    Returns:
        tuple: (bool, str) - (success status, message)
    """
    try:
        # Initialize GCS client with increased timeout
        storage_client = auth_gc_storage()

        # Check if bucket exists
        if not storage_client.lookup_bucket(bucket_name):
            return False, f"Bucket {bucket_name} not found"

        # Get bucket
        bucket = storage_client.bucket(bucket_name)

        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            # Create temporary file path
            temp_model_path = os.path.join(temp_dir, model_name)

            # Save model locally first
            model.save(temp_model_path, save_format='h5')

            # Configure retry logic
            retry_config = retry.Retry(
                initial=1.0,  # Initial delay in seconds
                maximum=60.0,  # Maximum delay in seconds
                multiplier=2.0,  # Multiplier for exponential backoff
                deadline=timeout,  # Total timeout
                predicate=retry.if_exception_type(
                    TimeoutError,
                    ConnectionError,
                    ConnectionAbortedError
                )
            )

            # Upload to GCS with retry logic
            blob = bucket.blob(model_name)

            # Set chunk size to 5MB for smaller models
            blob.chunk_size = 5 * 1024 * 1024  # 5MB in bytes

            for attempt in range(num_retries):
                try:
                    blob.upload_from_filename(
                        temp_model_path,
                        timeout=timeout,
                        retry=retry_config
                    )
                    gcs_uri = f"gs://{bucket_name}/{model_name}"
                    return True, f"Model successfully saved to {gcs_uri}"
                except Exception as e:
                    if attempt == num_retries - 1:  # Last attempt
                        raise
                    print(f"Attempt {attempt + 1} failed, retrying...")
                    continue

    except Exception as e:
        return False, f"Error saving model to GCS: {str(e)}"

In [61]:
save_model_to_gcs(model, model_bucket)

Running in local environment.
Authenticated successfully with GCS! ✅


(True,
 'Model successfully saved to gs://podcast-ad-skipper-model/latest_trained_model.h5')

In [62]:
# Took 10 m and 9.5 s to save the model trained with 5 clips' worth of data
# Will the model be heavier and hence take longer to save if so?

In [69]:
gcs_uri = f"gs://{BUCKET_NAME_MODEL}/latest_trained_model.h5"

In [70]:
model = download_model_from_gcs(gcs_uri)

Running in local environment.
Authenticated successfully with GCS! ✅

❌ No model found in GCS bucket podcast-ad-skipper


In [79]:
def download_model_from_gcs(bucket_name, model_name="latest_trained_model.h5"):
    """
    Download a TensorFlow model from Google Cloud Storage.

    Args:
        bucket_name: Name of the GCS bucket
        model_name: Name of the model file

    Returns:
        model: TensorFlow model or None if download fails
    """
    try:
        # Initialize GCS client
        storage_client = auth_gc_storage()

        # Check if bucket exists
        if not storage_client.lookup_bucket(bucket_name):
            return None, f"Bucket {bucket_name} not found"

        # Get bucket
        bucket = storage_client.bucket(bucket_name)

        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            # Create temporary file path
            temp_model_path = os.path.join(temp_dir, model_name)

            # Download file from GCS
            blob = bucket.blob(model_name)
            blob.download_to_filename(temp_model_path)

            # Load the model using TensorFlow
            model = tf.keras.models.load_model(temp_model_path)

            return model, f"Model successfully downloaded from gs://{bucket_name}/{model_name}"

    except Exception as e:
        return None, f"Error downloading model from GCS: {str(e)}"

In [80]:
model = download_model_from_gcs(BUCKET_NAME_MODEL, model_name="latest_trained_model.h5")

Running in local environment.
Authenticated successfully with GCS! ✅


In [81]:
model

(<Sequential name=sequential_1, built=True>,
 'Model successfully downloaded from gs://podcast-ad-skipper-model/latest_trained_model.h5')

In [82]:
# Took 2 m and 41 s to download the model above